# Notebook 1: Treinamento do Detector de Placas Brasileiras com YOLOv8

## 1. Objetivo

O objetivo deste notebook é treinar um modelo de detecção de objetos **YOLOv8** para identificar e localizar placas de veículos brasileiros em imagens. Utilizaremos um dataset customizado, já pré-formatado no padrão YOLO.

## 2. Plano de Ação

1.  **Configurar o Ambiente:** Instalar a biblioteca `ultralytics`, verificar a disponibilidade da GPU e importar as bibliotecas necessárias.
2.  **Upload e Extração dos Dados:** Fazer o upload do arquivo `.zip` contendo nosso dataset (`train/`, `test/`, `valid/` e `data.yaml`) e extraí-lo no ambiente do Colab.
3.  **Iniciar o Treinamento:** Executar o comando de treinamento do YOLOv8, especificando o modelo base e o arquivo de configuração `data.yaml`.
4.  **Salvar e Avaliar:** Salvar o nosso modelo treinado para uso futuro.

In [ ]:
# ==============================================================================
# CÉLULA DE CÓDIGO 1: SETUP DO AMBIENTE E UPLOAD DOS DADOS
# ==============================================================================

# 1. Instalação da biblioteca do YOLOv8
# O '-q' (quiet) é para uma instalação mais limpa, sem muita poluição visual.
!pip install ultralytics -q

# 2. Importação e verificação do ambiente
import ultralytics
import os
from google.colab import files

ultralytics.checks()

# O comando acima deve confirmar que a GPU foi encontrada.
# Lembrete: No Colab, vá em "Ambiente de execução" -> "Alterar tipo de ambiente" e selecione uma GPU (T4 é uma ótima opção).


# 3. Upload do arquivo .zip com o dataset
print("\nPor favor, selecione o arquivo .zip do seu dataset para fazer o upload.")
uploaded = files.upload()

# Pega o nome do arquivo que foi enviado (deve haver apenas um)
zip_filename = next(iter(uploaded))

print(f"\nArquivo '{zip_filename}' recebido com sucesso!")


# 4. Extração do conteúdo do .zip
print(f"Extraindo o conteúdo de '{zip_filename}'...")

# O '-o' sobrescreve arquivos existentes sem perguntar, e o '-q' opera no modo silencioso.
!unzip -o -q "{zip_filename}"

print("\n✅ Extração concluída!")
print("Seu dataset está pronto para o treinamento. As pastas 'train', 'test', 'valid' e o arquivo 'data.yaml' devem estar no diretório principal.")

# Opcional: Listar os arquivos para confirmar que tudo foi extraído corretamente
print("\nConteúdo do diretório atual:")
!ls

Ultralytics 8.3.205 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.4/112.6 GB disk)

Por favor, selecione o arquivo .zip do seu dataset para fazer o upload.


Saving placa.v5-placa_final_320.yolov8.zip to placa.v5-placa_final_320.yolov8.zip

Arquivo 'placa.v5-placa_final_320.yolov8.zip' recebido com sucesso!
Extraindo o conteúdo de 'placa.v5-placa_final_320.yolov8.zip'...

✅ Extração concluída!
Seu dataset está pronto para o treinamento. As pastas 'train', 'test', 'valid' e o arquivo 'data.yaml' devem estar no diretório principal.

Conteúdo do diretório atual:
data.yaml			     README.dataset.txt   test	 valid
placa.v5-placa_final_320.yolov8.zip  README.roboflow.txt  train


## 2. Treinamento do Modelo e Extração dos Resultados

Esta célula irá realizar o fluxo completo de treinamento e documentação do nosso modelo YOLOv8.

**O processo executado será:**
1.  **Treinamento Inteligente:** O comando `yolo` iniciará o treinamento a partir do modelo `yolov8s.pt`, usando nosso `data.yaml`. O treinamento rodará por no máximo 100 épocas, mas irá parar automaticamente se a performance não melhorar por 20 épocas (`patience=20`).
2.  **Extração Automática:** Assim que o treinamento for concluído, um script Python será executado para ler os arquivos de resultado gerados pelo YOLOv8 (métricas, hiperparâmetros).
3.  **Geração do Sumário:** Todas as informações serão consolidadas em um arquivo `training_summary.json`.
4.  **Download:** O download do arquivo JSON será iniciado automaticamente.

In [ ]:
# ==============================================================================
# CÉLULA ÚNICA: TREINAMENTO E EXTRAÇÃO DE RESULTADOS
# ==============================================================================

# Parte 1: Treinamento do Modelo YOLOv8
# ------------------------------------------------------------------------------
# Esta seção executa o treinamento. O script irá pausar aqui até que o
# treinamento seja concluído ou interrompido pelo Early Stopping.

print("--- INICIANDO O TREINAMENTO DO MODELO YOLOv8 ---")
!yolo task=detect mode=train model=yolov8s.pt data='data.yaml' epochs=300 imgsz=640 patience=20 name='plate_detector_v1'
print("\n--- TREINAMENTO CONCLUÍDO ---")


# Parte 2: Extração dos Resultados para JSON
# ------------------------------------------------------------------------------
# Esta seção será executada AUTOMATICAMENTE após a conclusão do treinamento.

print("\n--- INICIANDO A EXTRAÇÃO DOS RESULTADOS ---")
import json
import pandas as pd
import yaml
import os
from google.colab import files

# O nome da pasta é o que definimos no argumento 'name' do comando de treino
run_name = 'plate_detector_v1'
run_path = f'runs/detect/{run_name}/'

# Verificando se a pasta de resultados existe
if not os.path.exists(run_path):
    print(f"ERRO: A pasta de resultados '{run_path}' não foi encontrada.")
    print("O treinamento pode ter falhado. Verifique os logs acima.")
else:
    # Carregar os hiperparâmetros usados no treino
    with open(os.path.join(run_path, 'args.yaml'), 'r') as f:
        hyperparams = yaml.safe_load(f)
    print("Hiperparâmetros do treino carregados...")

    # Carregar o histórico de métricas de cada época
    results_df = pd.read_csv(os.path.join(run_path, 'results.csv'))
    results_df.columns = results_df.columns.str.strip()
    training_history = results_df.to_dict(orient='records')
    print("Histórico de treinamento carregado...")

    # Encontrar as métricas do melhor modelo
    best_epoch_data = results_df.loc[results_df['metrics/mAP50-95(B)'].idxmax()]

    best_model_metrics = {
        'epoch': int(best_epoch_data['epoch']),
        'precision(B)': best_epoch_data['metrics/precision(B)'],
        'recall(B)': best_epoch_data['metrics/recall(B)'],
        'mAP50(B)': best_epoch_data['metrics/mAP50(B)'],
        'mAP50-95(B)': best_epoch_data['metrics/mAP50-95(B)']
    }
    print("Métricas do melhor modelo extraídas...")

    # Consolidar tudo em um único dicionário
    training_summary = {
        'project_name': run_name,
        'model_path': os.path.join(run_path, 'weights/best.pt'),
        'hyperparameters': hyperparams,
        'best_model_metrics': best_model_metrics,
        'training_history_per_epoch': training_history
    }

    # Salvar e baixar o arquivo JSON
    summary_filename = f'{run_name}_summary.json'
    with open(summary_filename, 'w') as f:
        json.dump(training_summary, f, indent=4)

    print(f"\n✅ Sumário do treinamento salvo em: {summary_filename}")

    # Iniciar o download
    print("Iniciando o download do arquivo JSON...")
    files.download(summary_filename)

--- INICIANDO O TREINAMENTO DO MODELO YOLOv8 ---
Ultralytics 8.3.205 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=plate_detector_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patien

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3. Comprimi os outputs para Download.

In [ ]:
import os

# O comando que você quer executar, mas como uma string
comando = "tar -czvf runs.tar.gz runs/"

# O os.system executa o comando no terminal
os.system(comando)

print("Comando executado!")

Comando executado!
